##### Copyright 2021 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Lector de Apache ORC

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/orc"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/io/tutorials/orc.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/io/tutorials/orc.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/io/tutorials/orc.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

## Descripción general

Apache ORC es un formato de almacenamiento en columnas popular. El paquete tensorflow-io proporciona una implementación predeterminada para leer archivos [Apache ORC](https://orc.apache.org/).

## Preparación

Instale los paquetes necesarios y reinicie el tiempo de ejecución


In [2]:
!pip install tensorflow-io

In [3]:
import tensorflow as tf
import tensorflow_io as tfio

### Descargue un archivo de conjunto de datos de muestra en ORC

El conjunto de datos que usará es el [conjunto de datos Iris](https://archive.ics.uci.edu/ml/datasets/iris) de la UCI. El conjunto de datos contiene 3 clases de 50 instancias cada una, donde cada clase se refiere a un tipo de planta de iris. Tiene 4 atributos: (1) largo del sépalo, (2) ancho del sépalo, (3) largo del pétalo, (4) ancho del pétalo y la última columna contiene la etiqueta de clase.

In [4]:
!curl -OL https://github.com/tensorflow/io/raw/master/tests/test_orc/iris.orc
!ls -l iris.orc

## Crear un conjunto de datos a partir del archivo

In [35]:
dataset = tfio.IODataset.from_orc("iris.orc", capacity=15).batch(1)

Examine el conjunto de datos:

In [42]:
for item in dataset.take(1):
    print(item)


Veamos un ejemplo de principio a fin de entrenamiento del modelo tf.keras con un conjunto de datos de ORC basado en un conjunto de datos iris.

### Preprocesamiento de los datos

Configure qué columnas son características y qué columna es etiqueta:

In [47]:
feature_cols = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
label_cols = ["species"]

# select feature columns
feature_dataset = tfio.IODataset.from_orc("iris.orc", columns=feature_cols)
# select label columns
label_dataset = tfio.IODataset.from_orc("iris.orc", columns=label_cols)

Una función útil para asignar especies a números flotantes para el entrenamiento de modelos:

In [48]:
vocab_init = tf.lookup.KeyValueTensorInitializer(
    keys=tf.constant(["virginica", "versicolor", "setosa"]),
    values=tf.constant([0, 1, 2], dtype=tf.int64))
vocab_table = tf.lookup.StaticVocabularyTable(
    vocab_init,
    num_oov_buckets=4)

In [49]:
label_dataset = label_dataset.map(vocab_table.lookup)
dataset = tf.data.Dataset.zip((feature_dataset, label_dataset))
dataset = dataset.batch(1)

def pack_features_vector(features, labels):
    """Pack the features into a single array."""
    features = tf.stack(list(features), axis=1)
    return features, labels

dataset = dataset.map(pack_features_vector)

## Construir, compilar y entrenar el modelo

¡Finalmente, está todo listo para construir el modelo y entrenarlo! Construirá un modelo keras de 3 capas para predecir la clase de planta de iris a partir del conjunto de datos que acaba de procesar.

In [50]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(
            10, activation=tf.nn.relu, input_shape=(4,)
        ),
        tf.keras.layers.Dense(10, activation=tf.nn.relu),
        tf.keras.layers.Dense(3),
    ]
)

model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])
model.fit(dataset, epochs=5)